# 実習07-2 強化学習の体験（CartPole）
---
今回は、強化学習の体験として、Gymnasium（旧OpenAI Gym）で提供されているCartPoleを取り上げて、強化学習について理解を深める。

## 1. ゲームの作成
まずは、Gymnasiumをインストールする。
```
# gymnasiumのインストール
!pip install gymnasium
```
インストール後に、インポートしてゲームを作成する。今回はCartPole-v1を使用する。
[CartPole-v1](https://gymnasium.farama.org/environments/classic_control/cart_pole/)
```
# ゲームの作成（Warningが出るが、とりあえずそのままで）
import gymnasium as gym
env = gym.make("CartPole-v1")
```

In [1]:
# gymnasiumのインストール
!pip install gymnasium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 958.1/958.1 kB 13.5 MB/s eta 0:00:00


In [2]:
# ゲームの作成
import gymnasium as gym
env = gym.make("CartPole-v1")

## 2. ゲームの確認
実際にゲームを行って確認してみる。
まずは1回だけゲームを行ってみよう。

```
# ゲームの確認
for i in range(1):  # 1回ゲーム（あとで10回に変更）
    env.reset()     # ゲームの初期化
    # 1ゲームの最大回数 200stepまで実行。
    steps = 200
    for step in range(steps):
        # アクションを選択
        action = env.action_space.sample() # ランダムに左か右へ
        # 1 step 実行する。
        observation, reward, terminated, truncated, info = env.step(action)
        # action、observe, reward、doneを表示してみる（後でコメントアウト）
        print(action, observe, reward, done)

        # 終了判定がTrueなら1ゲーム終了
        if done:
            print("done!",step,"steps")
            break
        if step == steps - 1:
            print("GOAL!")
```

In [5]:
# ゲームの確認
for i in range(1):  # 1回ゲーム（あとで10回に変更）
    env.reset()     # ゲームの初期化
    # 1ゲームの最大回数 200stepまで実行。
    steps = 200
    for step in range(steps):
        # アクションを選択
        action = env.action_space.sample() # ランダムに左か右へ
        # 1 step 実行する。
        observe, reward, done, truncated, info = env.step(action)
        # action、observe, reward、doneを表示してみる（後でコメントアウト）
        print(action, observe, reward, done)

        # 終了判定がTrueなら1ゲーム終了
        if done:
            print("done!",step,"steps")
            break
        if step == steps - 1:
            print("GOAL!")

1 [-0.03592152  0.222312   -0.02459346 -0.26226318] 1.0 False
1 [-0.03147528  0.41777623 -0.02983872 -0.5626006 ] 1.0 False
0 [-0.02311976  0.22308542 -0.04109073 -0.27946565] 1.0 False
0 [-1.8658049e-02  2.8572988e-02 -4.6680048e-02 -2.0577883e-05] 1.0 False
1 [-0.01808659  0.22433224 -0.04668046 -0.30705854] 1.0 False
1 [-0.01359994  0.4200872  -0.05282163 -0.6140902 ] 1.0 False
0 [-0.0051982   0.2257416  -0.06510343 -0.3385007 ] 1.0 False
1 [-0.00068337  0.4217266  -0.07187345 -0.6509824 ] 1.0 False
0 [ 0.00775116  0.22767542 -0.08489309 -0.38176966] 1.0 False
1 [ 0.01230467  0.42389375 -0.09252848 -0.6999658 ] 1.0 False
0 [ 0.02078255  0.23016807 -0.10652781 -0.43778494] 1.0 False
0 [ 0.02538591  0.03670252 -0.1152835  -0.18049161] 1.0 False
0 [ 0.02611996 -0.15659723 -0.11889333  0.07371505] 1.0 False
1 [ 0.02298802  0.04001079 -0.11741903 -0.25398737] 1.0 False
0 [ 0.02378823 -0.15325594 -0.12249878 -0.00052538] 1.0 False
0 [ 0.02072311 -0.34642756 -0.12250929  0.2511361 ] 1.0 Fa

**[課題]action（行動）、observation（観測、状態）、reward（報酬）について、表示結果から分かることや考えられること、調べたことなどを書いてください。（分かる範囲で3つぐらいでOK）**

*  行動で学習のステップを設置
*  観測や状態などのデータで報酬を予測
*  予測した報酬で学習するデータを生成

ヒント
*  actionは左に押すか右に押すか
*  observeで表示されるものは、[cartの位置,cartの速度, Poleの角度、Poleの角速度]


## 3. Q学習を使用した強化学習の体験

### 3-1. 状態の離散化とQテーブルの作成

多くの強化学習では、状態と行動は離散値であり、
各状態の時に各行動をとった時の価値関数Qの値を表で管理することが多い。
今回は状態が連続値であるが、numpyを使用し、区間で区切って離散値にする。

```
#離散化（np.digitize使用）
np.digitize(値, bins=[どこで区切るか])
```
```
#binの作成（np.linspaceで等差数列を作る）
np.linspace(最小値, 最大値, 値の数)
```


In [6]:
# numpyをインポートし、npという名前で使えるようにしておく
import numpy as np

In [7]:
# linspaceの動作確認（記入済み）
num = 4
np.linspace(-3, 3, num+1)[1:-1]

array([-1.5,  0. ,  1.5])

In [8]:
# np.digitizeの動作確認（記入済み）
np.digitize([-2,-1,1,2], bins=np.linspace(-3, 3, num+1)[1:-1])

array([0, 1, 2, 3])

Qテーブルを作成する。
Q値は、Q(s, a)とし、状態sの時に行動aをとった時のQ値であるため
各状態と行動の次元（5次元）を持ち、それぞれの状態数と行動数の分だけ数値がある。最初にすべて0で初期化して表を作成する。

In [24]:
# 離散化の分割数（記入済み）
NUM = 6
import numpy as np
q = np.zeros((NUM,NUM,NUM,NUM,2))

次に、カートとポールの状態をデジタル化する関数を作成しておく。

In [25]:
# デジタル化する関数（記入済み）
POS_MIN, POS_MAX = -2.4, 2.4
VEL_MIN, VEL_MAX = -3.0, 3.0
ANG_MIN, ANG_MAX = -0.5, 0.5
ANGVEL_MIN, ANGVEL_MAX = -2.0, 2.0

def state2digit(state):
  pos, vel, ang, angvel = state
  return (np.digitize(pos, bins=np.linspace(POS_MIN, POS_MAX, NUM+1)[1:-1]),
        np.digitize(vel, bins=np.linspace(VEL_MIN, VEL_MAX, NUM+1)[1:-1]),
        np.digitize(ang, bins=np.linspace(ANG_MIN, ANG_MAX, NUM+1)[1:-1]),
        np.digitize(angvel, bins=np.linspace(ANGVEL_MIN, ANGVEL_MAX, NUM+1)[1:-1]))


### 3-2. 行動の決定とQ値の更新
行動と状態の更新は以下のようになる。

*   現在の状態における各行動のQ値を比較し、基本的に高い方の行動をとる。（今回はεの確率で違う方の行動をとる）
*   行動をとった後の環境の状態を取得する。
*   次の状態の各行動のQ値を使用して、前の状態のQ値を更新する。

これを上記のゲームの中に入れて実行する。

In [70]:
# パラメータ設定（記入済み）
EPSILON = 0.1 # ε-greedy法のε
ALPHA = 0.5 #学習率
GAMMA = 0.8 # 割引率

In [64]:
# アクションの選択：ε-greedy（記入済み）
def get_action(q, state):
    if EPSILON <= np.random.uniform(0, 1):
        next_action = np.argmax(q[state])
    else:
        next_action = np.random.choice([0, 1])
    return next_action

**[課題] プログラム中にQ値の更新式を書き、プログラムを完成させて実行する。**
env.resetのseedには学籍番号を指定すること。

In [71]:
# ゲームの確認
q = np.zeros((NUM,NUM,NUM,NUM,2)) # やり直すときはQをリセットする
GAMES = 2000
for i in range(GAMES):  # 繰り返し実行しながら学習
    observation_old,_ = env.reset(seed=2220042)     # ゲームの初期化
    observation_old = state2digit(observation_old)  # 状態の離散化
    # 1ゲームの最大回数 200stepまで実行。
    steps = 200
    for step in range(steps):
        # 行動を選択する
        action = get_action(q, observation_old)
        # 1 step 実行する。
        observation, reward, terminated, truncated, _ = env.step(action)
        # 倒れるとき（terminated==True）でもreward=1.0なので、reward=-1.0にしてみる
        if terminated: reward = -1.0
        # Q値を更新する（更新式に当てはめるだけ）
        observation = state2digit(observation) #状態の離散化
        next_Max_Q=max(q[observation]) # 計算に使用するQ値の最大値
        ######## ここを作成する ########
        q[observation_old][action]=q[observation_old][action]+ALPHA*(reward+GAMMA*next_Max_Q-q[observation_old][action])

        # 現在の状態を保存する
        observation_old = observation

        # 終了判定がTrueなら1ゲーム終了
        if terminated:
            print("done!",step,"steps")
            break
        if step == steps - 1 or truncated:
            print("GOAL!")

done! 9 steps
done! 28 steps
done! 9 steps
done! 9 steps
done! 36 steps
done! 29 steps
done! 9 steps
done! 30 steps
done! 32 steps
done! 9 steps
done! 17 steps
done! 22 steps
done! 12 steps
done! 32 steps
done! 34 steps
done! 63 steps
done! 54 steps
done! 62 steps
done! 30 steps
done! 32 steps
done! 27 steps
done! 60 steps
done! 59 steps
done! 34 steps
done! 18 steps
done! 18 steps
done! 18 steps
done! 18 steps
done! 18 steps
done! 18 steps
done! 19 steps
done! 18 steps
done! 15 steps
done! 18 steps
done! 16 steps
done! 14 steps
done! 12 steps
done! 27 steps
done! 44 steps
done! 71 steps
done! 49 steps
done! 30 steps
done! 60 steps
done! 56 steps
done! 63 steps
done! 19 steps
done! 17 steps
done! 49 steps
done! 16 steps
done! 12 steps
done! 45 steps
done! 45 steps
done! 32 steps
done! 33 steps
done! 79 steps
done! 17 steps
done! 20 steps
done! 112 steps
done! 18 steps
done! 44 steps
done! 150 steps
done! 58 steps
done! 78 steps
done! 128 steps
done! 165 steps
done! 59 steps
done! 58 st

### 3-3.ゲームの実施
得られたQ値を使用してゲームを実施する。ここでは、常にQ値が高い行動をとるものとする。

In [72]:
# ゲームの実施（記入済み）
for i in range(10):  # 10回ゲーム（10エピソード実行）
    observation,_ = env.reset()     # ゲームの初期化
    observation = state2digit(observation)  # 状態の離散化
    # 1ゲームの最大回数 200stepまで実行。
    steps = 200
    for step in range(steps):
        # アクションを選択
        action = np.argmax(q[observation])
        # 1 step 実行する。
        observation, reward, terminated, truncated,_ = env.step(action)
        observation = state2digit(observation)  # 状態の離散化

        # 終了判定がTrueなら1ゲーム終了
        if terminated:
            print("done!",step,"steps")
            break
        if step == steps - 1 or truncated:
            print("GOAL!", step, "steps")

done! 157 steps
GOAL! 199 steps
GOAL! 199 steps
GOAL! 199 steps
done! 137 steps
done! 141 steps
GOAL! 199 steps
done! 147 steps
GOAL! 199 steps
GOAL! 199 steps


**[課題]行動をランダムにした場合とQ学習の場合で、ゲームを行った結果を比較して分かることを書いてください。**

* **Q学習の学習エピソード数：[　2000　]**
* **比較して分かること：[　よくなるポリシーをもらう　]**

**※ 学習がうまくいくかどうかは、様々なパラメータとも関係する。学習してもランダムな場合とあまり変わらない時は、NUM、学習回数、ALPHA、GAMMA、rewardなどを変更してみるといいかもしれない。**

# 提出について

*   テキストブロックへの追記（CartPole-v1の行動、観測、報酬について、ランダムとQ学習の比較結果）
*   コードブロックの記入と実行（特にQ学習の更新式）

が確認出来たら、**教員に結果（パラメータの説明、最後の比較結果）を確認してもらうこと。** その後、ファイルが保存されているかを確認し、「ファイル＞ダウンロード＞.ipynbをダウンロード」を順にクリックして.ipynbファイルをダウンロードする。

ダウンロードしたipynbファイルを指定の場所に提出してください。
